In [8]:
#Auxiliares
import os
import pickle
from time import time
from datetime import datetime

#Dados
import pandas as pd


#Machine learning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report

seed = 42

In [9]:
#Variables
base_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/03. Datasets/Atendimento-Unbalanced-Binary'
save_path = 'output'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='ansi')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='ansi')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='ansi')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='ansi')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train.Atendimento.value_counts())
print(y_test.Atendimento.value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       '58home-response', '59money-response', '60relig-response',
       '61death-response', '62assent-response', '63nonfl-response',
       '64filler-response', 'Clareza', 'Atendimento', 'tempo_resposta'],
      dtype='object', length=134)
(31586, 134)
2    21682
0     9904
Name: Atendimento, dtype: int64
2    9338
0    4200
Name: Atendimento, dtype: int64


In [10]:
# Define pipeline and GridSearch CV
pipelineClf = Pipeline([                
    ('clf', LogisticRegression(random_state=seed, n_jobs=6, class_weight='balanced'))
])

parameters = {    
    'clf__C': (0.0001, 0.001, 0.01, 0.1, 1, 10),
}

# Define grid search
grid_search_tempo = GridSearchCV(pipelineClf,
                               parameters,
                               cv=10,
                               scoring='f1_macro',
                               n_jobs=6,
                               verbose=10
                               )

In [11]:
print("Executando Gridsearch para baseline tempo - Classe Atendimento")

now = str(datetime.now()).split('.')[0].replace('-', '_').replace(' ', '_').replace(':', '_')
print(now)

t0 = time()
grid_search_tempo.fit(X_train[['tempo_resposta']], y_train.Atendimento)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_tempo.best_score_)
print("Best parameters set:")
best_parameters = grid_search_tempo.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Executando Gridsearch para baseline tempo - Classe Atendimento
2020_05_19_23_17_07
Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.7s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.0s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  56 out of  60 | elapsed:    3.8s remaining:    0.2s


done in 4.323s
Best score: 0.533
Best parameters set:
	clf__C: 0.0001


[Parallel(n_jobs=6)]: Done  60 out of  60 | elapsed:    4.1s finished


In [12]:
# Saving Model
f_save = now + '_Atendimento_Baseline_Tempo_Binary_Unbalanced.sav'
pickle.dump(grid_search_tempo, open(os.path.join(save_path, f_save),'wb'))

# Saving Parameters
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write(f_save + '\n\n')
    for param_name in sorted(parameters.keys()):        
        f.write("%s: %r" % (param_name, best_parameters[param_name]))

# Validation

In [13]:
# Predictions
y_pred_train = grid_search_tempo.predict(X_train[['tempo_resposta']])
y_pred_test = grid_search_tempo.predict(X_test[['tempo_resposta']])

In [14]:
print('#'*50)
print('Report for TRAIN')
print('#'*50)
print(classification_report(y_train.Atendimento, y_pred_train))

print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(y_test.Atendimento, y_pred_test))

##################################################
Report for TRAIN
##################################################
              precision    recall  f1-score   support

           0       0.36      0.46      0.40      9904
           2       0.72      0.62      0.66     21682

    accuracy                           0.57     31586
   macro avg       0.54      0.54      0.53     31586
weighted avg       0.60      0.57      0.58     31586

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.35      0.45      0.40      4200
           2       0.72      0.62      0.67      9338

    accuracy                           0.57     13538
   macro avg       0.53      0.54      0.53     13538
weighted avg       0.60      0.57      0.58     13538



In [11]:
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TRAIN')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y_train.Atendimento, y_pred_train))
    
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TEST')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y_test.Atendimento, y_pred_test))